In [45]:
import numpy as np

In [46]:
data = """
널 품기 전 알지 못했다
내 머문 세상 이토록
찬란한 것을
작은 숨결로 닿은 사람
겁 없이 나를 불러준 사랑
몹시도 좋았다
너를 지켜보고 설레고
우습게 질투도 했던
평범한 모든 순간들이
캄캄한 영원
그 오랜 기다림 속으로
햇살처럼 니가 내렸다
널 놓기 전 알지 못했다
내 머문 세상 이토록
쓸쓸한 것을
고운 꽃이 피고 진 이 곳
다시는 없을 너라는 계절
욕심이 생겼다
너와 함께 살고 늙어가
주름진 손을 맞잡고
내 삶은 따뜻했었다고
단 한번 축복
그 짧은 마주침이 지나
빗물처럼 너는 울었다
한번쯤은 행복하고
싶었던 바람
너까지 울게 만들었을까
모두, 잊고 살아가라
내가 널, 찾을 테니
니 숨결, 다시 나를 부를 때
잊지 않겠다
너를 지켜보고 설레고
우습게 질투도 했던
니가 준 모든 순간들을
언젠가 만날
우리 가장 행복할 그날
첫눈처럼 내가 가겠다
너에게 내가 가겠다
"""


In [47]:
def data_preprocessing(data):
    tokens = data.split()
    vocab = list(set(tokens))
    vocab_size = len(vocab)

    word_to_ix = {word: i for i, word in enumerate(vocab)}
    ix_to_word = {i: word for i, word in enumerate(vocab)}

    return tokens, vocab_size, word_to_ix, ix_to_word


In [48]:
class RNN:
    def __init__(self, hidden_size, input_size, learning_rate, seq_len):
        #하이퍼 파라미터
        self.learning_rate = learning_rate
        self.hidden_size = hidden_size
        self.seq_len = seq_len
        self.hprev = np.zeros((hidden_size, 1)) # hidden_state t == 0 일때 즉, 초기 히든 스테이트

        self.input_size = input_size
        # input(x) -> input_Size x 1
        # hidden_state -> hidden_size x 1 
        self.U = np.random.randn(hidden_size, input_size) * 0.01 # U x input(x) 
        
        self.W = np.random.randn(hidden_size, hidden_size) * 0.01 # W x hidden_state + bw
        self.bw = np.zeros((hidden_size, 1))
        
        self.V = np.random.randn(input_size, hidden_size) * 0.01 # V x hidden_state + bv
        self.bv = np.zeros((input_size, 1))
        
        self.xs, self.hs, self.ps, self.ys = {}, {}, {}, {}

    
    def feedforward(self, inputs,targets):
        loss = 0
        self.hs[-1] = np.copy(self.hprev)
        
        for i in range(self.seq_len):
            self.xs[i] = np.zeros((self.input_size, 1))
            self.xs[i][inputs[i]] = 1  #input의 one hot encoding 
          
            self.hs[i] = np.tanh(np.dot(self.U, self.xs[i]) + np.dot(self.W, self.hs[i - 1]) + self.bw)
            
            self.ys[i] = np.dot(self.V, self.hs[i]) + self.bv
            
            self.ps[i] = np.exp(self.ys[i]) / np.sum(np.exp(self.ys[i]))  # softmax
          
            loss += -np.log(self.ps[i][targets[i]][0]) # loss function으로 cross_Entropy
        return loss
    def backward(self, targets):
    # Backward propagation through time (BPTT)
    # 처음에 모든 가중치들은 0으로 설정
        dV = np.zeros(self.V.shape)
        dW = np.zeros(self.W.shape)
        dU = np.zeros(self.U.shape)
        dbv = np.zeros(self.bv.shape)
        dbw = np.zeros(self.bw.shape)
        for i in range(self.seq_len)[::-1]:
            output = np.zeros((self.input_size, 1))
            output[targets[i]] = 1
            self.ps[i] = self.ps[i] - output.reshape(-1, 1)
        # 매번 i스텝에서 dL/dVi를 구하기
            dV_step_i = self.ps[i] @ (self.hs[i]).T  # (y_hat - y) @ hs.T - for each step
            dbv = self.ps[i]
            dV = dV + dV_step_i  # dL/dVi를 다 더하기

        # 각i별로 V와 W를 구하기 위해서는
        # 먼저 공통적으로 계산되는 부분을 delta로 해서 계산해두고
        # 그리고 시간을 거슬러 dL/dWij와 dL/dUij를 구한 뒤
        # 각각을 합하여 dL/dW와 dL/dU를 구하고 
        # 다시 공통적으로 계산되는 delta를 업데이트

        # i번째 스텝에서 공통적으로 사용될 delta
            delta_recent = (self.V.T @ self.ps[i]) * (1 - self.hs[i] ** 2)

        # 시간을 거슬러 올라가서 dL/dW와 dL/dU를 구하
            for j in range(i + 1)[::-1]:
                dbw += delta_recent
                dW_ij = delta_recent @ self.hs[j - 1].T

                dW = dW + dW_ij

                dU_ij = delta_recent @ self.xs[j].reshape(1, -1)
                dU = dU + dU_ij

            # 그리고 다음번 j번째 타임에서 공통적으로 계산할 delta를 업데이트
                delta_recent = (self.W.T @ delta_recent) * (1 - self.hs[j - 1] ** 2)

            for d in [dU, dW, dV]:
                np.clip(d, -1, 1, out=d)
                
        self.W -= self.learning_rate * dW
        self.U -= self.learning_rate * dU
        self.V -= self.learning_rate * dV
        self.bv -= self.learning_rate * dbv
        self.bw -= self.learning_rate * dbw
        self.hprev = self.hs[len(inputs) - 1]
        
    def predict(self, word, length):
        
        x = np.zeros((self.input_size, 1))
        x[word_to_ix[word]] = 1
        ixes = []
        h = np.zeros((self.hidden_size,1))

        for t in range(length):
            h = np.tanh(np.dot(self.U, x) + np.dot(self.W, h))
            y = np.dot(self.V, h)
            p = np.exp(y) / np.sum(np.exp(y))    # softmax
            ix = np.argmax(p)                    # 가장 높은 확률의 index return
            x = np.zeros((self.input_size, 1))        # 다음 input one hot encoding
            x[ix] = 1
            ixes.append(ix) 
            
        pred_words = word + ' '+' '.join(ix_to_word[i] for i in ixes)
        return pred_words
        

In [49]:
tokens, vocab_size, word_to_ix, ix_to_word = data_preprocessing(data)
seq_len=1
h_size=100
learning_rate = 0.01
epochs = 1000
rnn = RNN(h_size, vocab_size, learning_rate, seq_len)

In [50]:
p=0
for epoch in range(epochs):
    
    for p in range(len(tokens)-seq_len):
        inputs = [word_to_ix[tok] for tok in tokens[p:p + seq_len]]
        targets = [word_to_ix[tok] for tok in tokens[p + 1:p + seq_len + 1]]

        loss = rnn.feedforward(inputs,targets)
        rnn.backward(targets)

    if epoch % 100 == 0:
        print(f'epoch {epoch}, loss: {loss}')


epoch 0, loss: 4.644721672329932
epoch 100, loss: 3.835155015840123
epoch 200, loss: 0.1022253581285526
epoch 300, loss: 0.01991212294776481
epoch 400, loss: 0.00975137488728447
epoch 500, loss: 0.006250863842611008
epoch 600, loss: 0.0045618762418998295
epoch 700, loss: 0.003580000333975143
epoch 800, loss: 0.002940009316026212
epoch 900, loss: 0.002490519774959017


In [56]:
rnn.predict("널",126)

'널 품기 전 알지 못했다 내 머문 세상 이토록 찬란한 것을 작은 숨결로 닿은 사람 겁 없이 나를 불러준 사랑 몹시도 좋았다 너를 지켜보고 설레고 우습게 질투도 했던 평범한 모든 순간들이 캄캄한 영원 그 오랜 기다림 속으로 햇살처럼 니가 내렸다 널 놓기 전 알지 못했다 내 머문 세상 이토록 쓸쓸한 것을 고운 꽃이 피고 진 이 곳 다시는 없을 너라는 계절 욕심이 생겼다 너와 함께 살고 늙어가 주름진 손을 맞잡고 내 삶은 따뜻했었다고 단 한번 축복 그 짧은 마주침이 지나 빗물처럼 너는 울었다 한번쯤은 행복하고 싶었던 바람 너까지 울게 만들었을까 모두, 잊고 살아가라 내가 널, 찾을 테니 니 숨결, 다시 나를 부를 때 잊지 않겠다 너를 지켜보고 설레고 우습게 질투도 했던 니가 준 모든 순간들을 언젠가 만날 우리 가장 행복할 그날 첫눈처럼 내가 가겠다 너에게 내가 가겠다'